In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
# import transformers
# nltk.download('punkt')
# nltk.download('stopwords')
stop = stopwords.words('english')
import re
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing, metrics
from lime import lime_text
import gensim
import gensim.downloader as gensim_api
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

In [2]:
all_wines = pd.read_csv('Data/winemag-data-130k-v2.csv')
all_wines.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
clean_wines = all_wines.drop(columns=['Unnamed: 0', 'taster_twitter_handle', 'region_2'])
clean_wines = clean_wines.dropna()
clean_wines.head()

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam


In [4]:
clean_wines['variety'] = clean_wines['variety'].replace(['Weissburgunder'], 'Chardonnay')
clean_wines['variety'] = clean_wines['variety'].replace(['Spatburgunder'], 'Pinot Noir')
clean_wines['variety'] = clean_wines['variety'].replace(['Grauburgunder'], 'Pinot Gris')
clean_wines['variety'] = clean_wines['variety'].replace(['Garnacha'], 'Grenache')
clean_wines['variety'] = clean_wines['variety'].replace(['Pinot Nero'], 'Pinot Noir')
clean_wines['variety'] = clean_wines['variety'].replace(['Alvarinho'], 'Albarino')

In [5]:
clean_wines['vintage'] = clean_wines['title'].str.extract('(\d\d\d\d)', expand=True)
clean_wines

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012
...,...,...,...,...,...,...,...,...,...,...,...,...
129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim,2010
129962,Italy,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,Kerin O’Keefe,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,Cusumano,2012
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,2013
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013


In [6]:
clean_wines['type'] = ['white' if x in ['Chardonnay', 'Riesling', 'Sauvignon Blanc', 'White Blend', 'Sparkling Blend', 'Pinot Gris',
                                        'Champagne Blend', 'GrÃ¼ner Veltliner', 'Pinot Grigio', 'Portuguese White', 'Viognier',
                                        'GewÃ¼rztraminer', 'GewÃ¼rztraminer'] else 'red' for x in clean_wines['variety']]
clean_wines


,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage,type
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,white
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,red
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011,red
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,red
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim,2010,white
129962,Italy,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,Kerin O’Keefe,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,Cusumano,2012,red
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,2013,white
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013,red


In [7]:
clean_wines['word_count'] = clean_wines['description'].str.lower().str.split().str.len()
clean_wines

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage,type,word_count
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,white,33
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,red,41
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011,red,42
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,red,31
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012,white,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim,2010,white,52
129962,Italy,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,Kerin O’Keefe,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,Cusumano,2012,red,37
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,2013,white,32
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013,red,43


In [8]:
clean_wines = clean_wines[clean_wines['vintage'].notna()]
clean_wines = clean_wines.copy()
clean_wines['vintage'] = clean_wines['vintage'].astype(int)
clean_wines['age'] = 2020 - clean_wines['vintage'] 
clean_wines

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage,type,word_count,age
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,white,33,7
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,red,41,8
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011,red,42,9
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,red,31,7
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012,white,30,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim,2010,white,52,10
129962,Italy,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,Kerin O’Keefe,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,Cusumano,2012,red,37,8
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,2013,white,32,7
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013,red,43,7


In [9]:
clean_wines = clean_wines.drop_duplicates('description')
clean_wines = clean_wines[pd.notnull(clean_wines.price)]

In [10]:
clean_wines.quantile(q=0.75, axis=0, numeric_only=True)

points          91.0
price           49.0
vintage       2014.0
word_count      49.0
age             10.0
Name: 0.75, dtype: float64

In [ ]:
clean_wines['points'].hist()

In [11]:
clean_wines['good_meh'] = clean_wines['points'].apply(lambda x: 'good' if x >= 91 else 'meh')
clean_wines

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage,type,word_count,age,good_meh
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,white,33,7,meh
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,red,41,8,meh
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011,red,42,9,meh
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,red,31,7,meh
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012,white,30,8,meh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim,2010,white,52,10,meh
129962,Italy,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,Kerin O’Keefe,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,Cusumano,2012,red,37,8,meh
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,2013,white,32,7,meh
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013,red,43,7,meh


In [12]:
good_wines = clean_wines[clean_wines['good_meh'] == 'good']
good_wines

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage,type,word_count,age,good_meh
123,Australia,The blend is roughly two-thirds Shiraz and one...,Parson's Flat,92,40.0,South Australia,Padthaway,Joe Czerwinski,Henry's Drive Vignerons 2006 Parson's Flat Shi...,Shiraz-Cabernet Sauvignon,Henry's Drive Vignerons,2006,red,36,14,good
144,US,"Thick and brooding, this dark, sweetly tannic ...",K Block,91,85.0,California,Spring Mountain District,Virginie Boone,Terra Valentine 2013 K Block Cabernet Sauvigno...,Cabernet Sauvignon,Terra Valentine,2013,red,37,7,good
145,US,"Cooked cranberry is spiced with anise, pepperc...",Guidotti Vineyard,91,64.0,California,Santa Lucia Highlands,Matt Kettmann,Testarossa 2013 Guidotti Vineyard Pinot Noir (...,Pinot Noir,Testarossa,2013,red,43,7,good
146,US,"There's a touch of hot asphalt, pencil lead an...",Family Reserve,91,68.0,California,Santa Ynez Valley,Matt Kettmann,Vincent Vineyards 2010 Family Reserve Cabernet...,Cabernet Sauvignon,Vincent Vineyards,2010,red,51,10,good
147,US,"Black cherry, black plum and black currant are...",Family Reserve,91,68.0,California,Santa Ynez Valley,Matt Kettmann,Vincent Vineyards 2012 Family Reserve Cabernet...,Cabernet Sauvignon,Vincent Vineyards,2012,red,55,8,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129928,Spain,This represents Torres' highest effort in Prio...,Salmos Perpetual,91,52.0,Catalonia,Priorat,Michael Schachner,Torres 2005 Salmos Perpetual Red (Priorat),Red Blend,Torres,2005,red,47,15,good
129930,Spain,"Arguably on the heavy and rich side, but if yo...",Crianza,91,26.0,Northern Spain,Ribera del Duero,Michael Schachner,Costaval 2005 Crianza (Ribera del Duero),Tempranillo Blend,Costaval,2005,red,48,15,good
129932,Argentina,Andeluna's top wines tend to be ripe and plump...,Pasionado,91,55.0,Mendoza Province,Uco Valley,Michael Schachner,Andeluna 2004 Pasionado Red (Uco Valley),Red Blend,Andeluna,2004,red,61,16,good
129936,France,A blend from several premier cru vineyards in ...,Premier Cru,91,66.0,Burgundy,Pommard,Roger Voss,Bouchard Père & Fils 2005 Premier Cru (Pommard),Pinot Noir,Bouchard Père & Fils,2005,red,61,15,good


In [13]:
clean_wines['char_count'] = clean_wines["description"].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
clean_wines['sentence_count'] = clean_wines["description"].apply(lambda x: len(str(x).split(".")))
clean_wines['avg_word_length'] = clean_wines['char_count'] / clean_wines['word_count']
clean_wines['avg_sentence_lenght'] = clean_wines['word_count'] / clean_wines['sentence_count']
clean_wines

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,vintage,type,word_count,age,good_meh,char_count,sentence_count,avg_word_length,avg_sentence_lenght
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,white,33,7,meh,167,3,5.060606,11.000000
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,red,41,8,meh,209,3,5.097561,13.666667
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011,red,42,9,meh,220,4,5.238095,10.500000
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,red,31,7,meh,151,3,4.870968,10.333333
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012,white,30,8,meh,131,4,4.366667,7.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129959,France,The granite soil of the Brand Grand Cru vineya...,Brand Grand Cru,90,57.0,Alsace,Alsace,Roger Voss,Cave de Turckheim 2010 Brand Grand Cru Pinot G...,Pinot Gris,Cave de Turckheim,2010,white,52,10,meh,237,5,4.557692,10.400000
129962,Italy,"Blackberry, cassis, grilled herb and toasted a...",Sàgana Tenuta San Giacomo,90,40.0,Sicily & Sardinia,Sicilia,Kerin O’Keefe,Cusumano 2012 Sàgana Tenuta San Giacomo Nero d...,Nero d'Avola,Cusumano,2012,red,37,8,meh,184,4,4.972973,9.250000
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,2013,white,32,7,meh,180,3,5.625000,10.666667
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013,red,43,7,meh,183,4,4.255814,10.750000


In [14]:
winery_counts = {'winery' : clean_wines['winery'].unique(), 
                 'winery_count' : clean_wines['winery'].value_counts() 

            }
winery_df = pd.DataFrame(data=winery_counts)
winery_df.reset_index(drop=True, inplace=True)
winery_df

,winery,winery_count
0,St. Julian,144
1,Sweet Cheeks,117
2,Tandem,103
3,Terre di Giurfo,100
4,Jean-Baptiste Adam,90
...,...,...
8286,Yacochuya,1
8287,Tollot Beaut et Fils,1
8288,Bottega,1
8289,Bodegas Eidosela,1


In [15]:
clean_wines = clean_wines.merge(winery_df, how='inner', on='winery')
clean_wines.sample(30)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,...,vintage,type,word_count,age,good_meh,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count
35932,US,This is a solidly built wine with ripe black-f...,Estate Grown,86,18.0,California,Mendocino,Jim Gordon,Barra of Mendocino 2013 Estate Grown Sangioves...,Sangiovese,...,2013,red,32,7,meh,159,3,4.968750,10.666667,3
12038,US,"From 100% estate-grown fruit, this lemony wine...",Estate,84,45.0,California,Napa Valley,Virginie Boone,Kuleto Estate 2013 Estate Chardonnay (Napa Val...,Chardonnay,...,2013,white,35,7,meh,176,3,5.028571,11.666667,14
6567,US,Sourced entirely from the biodynamic estate vi...,Bergström Vineyard,95,81.0,Oregon,Dundee Hills,Paul Gregutt,Bergström 2009 Bergström Vineyard Pinot Noir (...,Pinot Noir,...,2009,red,59,11,good,328,5,5.559322,11.800000,22
32933,Argentina,Woody aromas are obvious but not overdone as t...,Unánime,88,25.0,Mendoza Province,Mendoza,Michael Schachner,Mascota 2016 Unánime Chardonnay (Mendoza),Chardonnay,...,2016,white,40,4,meh,194,4,4.850000,10.000000,4
7300,France,This attractively soft wine is as gentle as th...,Aramis,85,10.0,France Other,Vin de France,Roger Voss,Famille Laplace 2016 Aramis Colombard-Sauvigno...,Colombard-Sauvignon Blanc,...,2016,red,33,4,meh,170,4,5.151515,8.250000,20
42976,France,Red cherry aromas lead to a wine that is curio...,Natur,82,18.0,Bordeaux,Bordeaux,Roger Voss,Château Penin 2013 Natur (Bordeaux),Merlot,...,2013,red,33,7,meh,160,3,4.848485,11.000000,1
43118,US,This is arguably the best of the current Agate...,Estate,88,24.0,Oregon,Rogue Valley,Paul Gregutt,Agate Ridge 2009 Estate Syrah (Rogue Valley),Syrah,...,2009,red,34,11,meh,176,4,5.176471,8.500000,1
3970,Spain,An elegant Rioja made in a more traditional st...,Castillo Ygay Gran Reserva Especial,90,54.0,Northern Spain,Rioja,Michael Schachner,Marqués de Murrieta 2000 Castillo Ygay Gran Re...,Tempranillo Blend,...,2000,red,62,20,meh,294,5,4.741935,12.400000,29
36677,France,"Tight and herbaceous, this nervy, crisp wine i...",La Grande Cuvée,86,14.0,Bordeaux,Bordeaux Blanc,Roger Voss,Dourthe 2013 La Grande Cuvée (Bordeaux Blanc),Sauvignon Blanc,...,2013,white,30,7,meh,140,3,4.666667,10.000000,3
44037,Italy,This opens with faint honeysuckle and white st...,Fontana,85,12.0,Veneto,Soave,Kerin O’Keefe,Tenuta Sant'Antonio 2016 Fontana (Soave),Garganega,...,2016,red,24,4,meh,138,3,5.750000,8.000000,1


In [16]:
country_counts = {'country' : clean_wines['country'].unique(),
             'country_count' : clean_wines['country'].value_counts()
            }
country_df = pd.DataFrame(data=country_counts)
country_df.reset_index(drop=True, inplace=True)
country_df

,country,country_count
0,US,23237
1,Spain,9408
2,Italy,6120
3,France,4337
4,Argentina,2579
5,Australia,1309
6,Canada,154


In [17]:
clean_wines = clean_wines.merge(country_df, how='inner', on='country')
clean_wines.sample(30)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,...,type,word_count,age,good_meh,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count,country_count
30456,Italy,"This fragrant, light-bodied and savory Sicilia...",Il Frappato,90,23.0,Sicily & Sardinia,Vittoria Frappato,Kerin O’Keefe,Valle dell'Acate 2013 Il Frappato (Vittoria F...,Frappato,...,red,49,7,meh,252,4,5.142857,12.250000,4,6120
1348,US,"Earthy and engaging, and a little bit rugged. ...",3 Vineyard,90,25.0,Oregon,Willamette Valley,Michael Schachner,Chehalem 1998 3 Vineyard Pinot Noir (Willamett...,Pinot Noir,...,red,40,22,meh,198,4,4.950000,10.000000,34,23237
34027,France,"The vineyard is high up the slope, giving this...",Sous le Puits Premier Cru,93,95.0,Burgundy,Puligny-Montrachet,Roger Voss,Louis Latour 2014 Sous le Puits Premier Cru (...,Chardonnay,...,white,42,6,good,189,5,4.500000,8.400000,38,4337
17475,US,"Floral and soft, this is an enjoyable well-cra...",Right Bank,88,50.0,California,Oak Knoll District,Virginie Boone,Materra Cunat Family Vineyards 2013 Right Bank...,Merlot,...,red,26,7,meh,135,3,5.192308,8.666667,4,23237
37759,France,"This light, fruity wine is a perfect summer ap...",Terra Santa,85,10.0,France Other,Ile de Beauté,Roger Voss,Union de Vignerons de l'Île de Beauté 2016 Ter...,Rosé,...,red,23,4,meh,117,4,5.086957,5.750000,7,4337
41783,France,"This elegant, lithe little wine is ready to dr...",Domaine de Clairfont,89,12.0,Rhône Valley,Vin de Pays de Vaucluse,Joe Czerwinski,Vignobles Guy Mousset et Fils 2015 Domaine de ...,Rhône-style Red Blend,...,red,28,5,meh,145,3,5.178571,9.333333,2,4337
41795,France,Old Merlot vines from a five-acre parcel have ...,Cuvée Vieilles Vignes,90,30.0,Bordeaux,Puisseguin Saint-Émilion,Roger Voss,Château Lanbersac 2012 Cuvée Vieilles Vignes ...,Merlot,...,red,40,8,meh,193,5,4.825000,8.000000,2,4337
15449,US,This medium-bodied wine is worth exploring. It...,Red Tail Vineyard,88,20.0,California,Lodi,Jim Gordon,LangeTwins 2014 Red Tail Vineyard Nero d'Avola...,Nero d'Avola,...,red,40,6,meh,215,4,5.375000,10.000000,5,23237
37406,France,This ripe and full-bodied wine has a wonderful...,Authentique,88,25.0,Burgundy,Pouilly-Fuissé,Roger Voss,Domaine Sangouard-Guyot 2013 Authentique (Pou...,Chardonnay,...,white,43,7,meh,199,4,4.627907,10.750000,8,4337
31555,Italy,"Pear, apple, hazelnut and crushed stone aromas...",Karneid,90,26.0,Northeastern Italy,Alto Adige,Kerin O’Keefe,Franz Gojer-Glögglhof 2014 Karneid Pinot Bianc...,Pinot Bianco,...,red,36,6,meh,194,4,5.388889,9.000000,3,6120


In [18]:
province_counts = {'province' : clean_wines['province'].unique(),
             'province_count' : clean_wines['province'].value_counts()
            }
province_df = pd.DataFrame(data=province_counts)
province_df.reset_index(drop=True, inplace=True)
province_df

,province,province_count
0,Michigan,12491
1,Oregon,5325
2,California,3423
3,Washington,2770
4,Virginia,2209
5,New York,1928
6,Idaho,1824
7,Texas,1574
8,Pennsylvania,1462
9,New Jersey,1383


In [19]:
clean_wines = clean_wines.merge(province_df, how='inner', on='province')
clean_wines.sample(30)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,...,word_count,age,good_meh,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count,country_count,province_count
40149,France,"The wine is perfumed and full. It is so soft, ...",Zee Original,84,12.0,Southwest France,Côtes du Lot,Roger Voss,Pelvillain Frères 2014 Zee Original Malbec (Cô...,Malbec,...,25,6,meh,118,5,4.720000,5.000000,4,4337,17
22661,US,"Spicy and fruity on the nose, with scents of b...",Estate Reserve,85,16.0,New York,Finger Lakes,Anna Lee C. Iijima,Stony Lonesome 2010 Estate Reserve Barbera (Fi...,Barbera,...,47,10,meh,228,3,4.851064,15.666667,6,23237,1928
3310,US,"Aged in 30% new French oak, this cuvée opens w...",Cyrus,91,36.0,Oregon,McMinnville,Paul Gregutt,Maysara 2012 Cyrus Pinot Noir (McMinnville),Pinot Noir,...,43,8,good,205,4,4.767442,10.750000,2,23237,5325
26652,Spain,Lemony and light on the bouquet. The palate is...,Calitja,88,15.0,Catalonia,Terra Alta,Michael Schachner,Celler La Bollidora 2009 Calitja Garnacha Blan...,Garnacha Blanca,...,48,11,meh,244,5,5.083333,9.600000,7,9408,391
14602,US,"A Sauvignon Blanc-based blend, this opens with...",Winemaker's Select,82,45.0,California,Napa Valley,Virginie Boone,Sterling 2013 Winemaker's Select White (Napa V...,White Blend,...,23,7,meh,112,2,4.869565,11.500000,2,23237,3423
29234,Italy,Fans of full-bodied reds should like this Cabe...,Excelsus,90,73.0,Tuscany,Sant'Antimo,Michael Schachner,Castello Banfi 1998 Excelsus Red (Sant'Antimo),Bordeaux-style Red Blend,...,47,22,meh,237,4,5.042553,11.750000,3,6120,254
47137,Canada,"Though pale in color, this is a tannic and fla...",Foxtrot Vineyard,90,50.0,British Columbia,Okanagan Valley,Paul Gregutt,Foxtrot 2013 Foxtrot Vineyard Pinot Noir (Okan...,Pinot Noir,...,34,7,meh,188,4,5.529412,8.500000,1,154,1
13159,US,A strong percentage of Petit Verdot (14%) mark...,Reserve,88,110.0,California,Rutherford,Virginie Boone,Round Pond 2011 Reserve Cabernet Sauvignon (Ru...,Cabernet Sauvignon,...,37,9,meh,184,2,4.972973,18.500000,3,23237,3423
33079,Italy,"Packaged in an extremely heavy bottle, this fe...",10 Anni Riserva,87,300.0,Veneto,Amarone della Valpolicella,Kerin O’Keefe,Villa Canestrari 2005 10 Anni Riserva (Amaron...,Red Blend,...,41,15,meh,207,3,5.048780,13.666667,4,6120,89
39594,France,"With its spice, white pear and nutmeg flavors,...",Le Jardin Philosophique,88,19.0,Southwest France,Pacherenc du Vic Bilh,Roger Voss,Alain Brumont 2006 Le Jardin Philosophique (P...,Petit Courbu,...,38,14,meh,191,3,5.026316,12.666667,18,4337,17


In [20]:
region_counts = {'region_1' : clean_wines['region_1'].unique(),
             'region_count' : clean_wines['region_1'].value_counts()
            }
region_df = pd.DataFrame(data=region_counts)
region_df.reset_index(drop=True, inplace=True)
region_df

,region_1,region_count
0,Lake Michigan Shore,2240
1,Michigan,1509
2,Old Mission Peninsula,1483
3,Leelanau Peninsula,1344
4,Willamette Valley,1178
...,...,...
982,Short Hills Bench,1
983,St. David's Bench,1
984,Prince Edward County,1
985,Okanagan Valley,1


In [21]:
clean_wines = clean_wines.merge(region_df, how='inner', on='region_1')
clean_wines.sample(30)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,...,age,good_meh,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count,country_count,province_count,region_count
31767,Italy,"Underbrush, scorched earth, ripe berry, blue f...",Arione,93,59.0,Piedmont,Barolo,Kerin O’Keefe,Gigi Rosso 2013 Arione (Barolo),Nebbiolo,...,7,good,315,6,5.625000,9.333333,6,6120,154,9
28396,Italy,"Aromas of ripe black-skinned fruit, new leathe...",Riserva,98,190.0,Tuscany,Brunello di Montalcino,Kerin O’Keefe,Conti Costanti 2010 Riserva (Brunello di Mont...,Sangiovese,...,10,good,268,6,5.702128,7.833333,2,6120,254,19
14461,US,"Tasty, generous and easy to enjoy, this full-b...",Old Vine,87,20.0,California,Lodi,Jim Gordon,Criss Cross 2012 Old Vine Zinfandel (Lodi),Zinfandel,...,8,meh,157,3,5.233333,10.000000,1,23237,3423,119
20233,US,Aromas of marionberry jam and smoke leap out o...,Boushey Vineyard,84,36.0,Washington,Yakima Valley,Sean P. Sullivan,Telaya 2015 Boushey Vineyard Malbec (Yakima Va...,Malbec,...,5,meh,192,3,4.571429,14.000000,7,23237,2770,48
14798,US,"Smoky, nutty aromas blend well with rich, bake...",Estate,88,29.0,California,Shenandoah Valley (CA),Jim Gordon,Wilderotter 2013 Estate Barbera (Shenandoah Va...,Barbera,...,7,meh,121,2,6.050000,10.000000,3,23237,3423,107
29566,Italy,"Aromas of sun-baked earth, pressed iris, wild ...",Riserva,90,19.0,Tuscany,Chianti Classico,Kerin O’Keefe,Capannelle 2012 Riserva (Chianti Classico),Sangiovese,...,8,meh,262,5,5.137255,10.200000,1,6120,254,18
44538,Argentina,"Mossy, foresty aromas are rugged and settle in...",Epica,87,11.0,Mendoza Province,Mendoza,Michael Schachner,San Pedro 2013 Epica Malbec (Mendoza),Malbec,...,7,meh,230,4,4.893617,11.750000,1,2579,9,1
12378,US,A tiny-production wine that is exuberantly bra...,Riverwash,93,35.0,California,Dry Creek Valley,Virginie Boone,Sequum 2012 Riverwash Zinfandel (Dry Creek Val...,Zinfandel,...,8,good,236,4,5.021277,11.750000,45,23237,3423,155
44592,Argentina,Green aromas of oregano and basil blend with t...,Reserva,82,10.0,Mendoza Province,Mendoza,Michael Schachner,Viñas de Vila 2009 Reserva Merlot (Mendoza),Merlot,...,11,meh,124,3,4.769231,8.666667,1,2579,9,1
9898,US,Estate-grown from a small family winery and en...,Reserve,90,65.0,California,Calistoga,Virginie Boone,Summers 2011 Reserve Cabernet Sauvignon (Calis...,Cabernet Sauvignon,...,9,meh,246,4,5.234043,11.750000,19,23237,3423,266


In [22]:
taster_counts = {'taster_name' : clean_wines['taster_name'].unique(),
             'taster_count' : clean_wines['taster_name'].value_counts()
            }
taster_df = pd.DataFrame(data=taster_counts)
taster_df.reset_index(drop=True, inplace=True)
taster_df

,taster_name,taster_count
0,Alexander Peartree,7433
1,Anna Lee C. Iijima,7022
2,Susan Kostrzewa,6065
3,Paul Gregutt,5978
4,Michael Schachner,5915
5,Joe Czerwinski,4068
6,Sean P. Sullivan,3013
7,Virginie Boone,2261
8,Jim Gordon,2119
9,Matt Kettmann,1490


In [23]:
clean_wines = clean_wines.merge(taster_df, how='inner', on='taster_name')
clean_wines.sample(30)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,...,good_meh,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count,country_count,province_count,region_count,taster_count
6056,US,"This classic Washington blend is 60% Merlot, 3...",Mésalliance,91,20.0,Washington,Columbia Valley (WA),Paul Gregutt,Andrew Rich 2005 Mésalliance Red (Columbia Val...,Red Blend,...,good,334,5,5.301587,12.600000,8,23237,2770,50,5978
31490,US,This upper-tier label shows condensed strawber...,Rancho La Vina,92,50.0,California,Sta. Rita Hills,Matt Kettmann,Potek 2013 Rancho La Vina Pinot Noir (Sta. Rit...,Pinot Noir,...,good,217,3,5.710526,12.666667,1,23237,3423,185,1490
10225,Spain,"Aromas of berry fruit, wet cement and chewing ...",Agostón,84,13.0,Northern Spain,Cariñena,Michael Schachner,Bodegas Paniza 2015 Agostón Tempranillo-Cabern...,Tempranillo-Cabernet Sauvignon,...,meh,169,3,4.970588,11.333333,16,9408,531,35,5915
16582,Australia,"Meaty and savory, this is surprisingly full-bo...",Under Woods Steels Creek,92,55.0,Victoria,Yarra Valley,Joe Czerwinski,Ben Haines 2013 Under Woods Steels Creek Syrah...,Syrah,...,good,175,3,5.645161,10.333333,20,1309,4,1,4068
17395,US,"The aromas of dark coffee, bittersweet chocola...",Reserve,92,45.0,Washington,Columbia Valley (WA),Sean P. Sullivan,Columbia Crest 2013 Reserve Cabernet Sauvignon...,Cabernet Sauvignon,...,good,208,3,5.073171,13.666667,14,23237,2770,50,3013
26563,US,Great aromatic complexity and richly layered f...,Ten Degrees,94,120.0,California,Anderson Valley,Jim Gordon,Goldeneye 2013 Ten Degrees Pinot Noir (Anderso...,Pinot Noir,...,good,239,4,5.431818,11.000000,4,23237,3423,297,2119
22536,US,This viscous wine offers an underlay of toasty...,Freestone Vineyards Estate Grown,93,55.0,California,Sonoma Coast,Virginie Boone,Joseph Phelps 2014 Freestone Vineyards Estate ...,Chardonnay,...,good,175,3,5.147059,11.333333,18,23237,3423,304,2261
43837,Italy,"This opens with aromas of bread crust, citrus ...",Vieris,89,47.0,Northeastern Italy,Friuli Isonzo,Kerin O’Keefe,Vie di Romans 2015 Vieris Sauvignon (Friuli Is...,Sauvignon,...,meh,194,3,4.850000,13.333333,4,6120,155,9,84
17819,US,Cabernet Sauvignon (84%) takes the lead on thi...,Arctos,91,43.0,Washington,Columbia Valley (WA),Sean P. Sullivan,Baer 2013 Arctos Red (Columbia Valley (WA)),Bordeaux-style Red Blend,...,good,240,3,5.217391,15.333333,6,23237,2770,50,3013
46490,France,Hay flowers and juicy yellow pear create an en...,Vorbourg Grand Cru,92,30.0,Alsace,Alsace,Anne Krebiehl MW,Dopff & Irion 2010 Vorbourg Grand Cru Pinot Gr...,Pinot Gris,...,good,307,6,5.203390,9.833333,13,4337,57,5,8


In [24]:
# winery_scores = {'winery' : clean_wines['winery'].unique(), 
#                  'winery_avg' : clean_wines[clean_wines['winery'].value_counts() 

#             }
# winery_df = pd.DataFrame(data=winery_counts)
# winery_df.reset_index(drop=True, inplace=True)
# winery_df

In [25]:
features = clean_wines[['price', 'age', 'word_count', 'char_count', 'sentence_count', 'avg_word_length', 
                        'avg_sentence_lenght', 'winery_count', 'country_count', 'province_count', 'region_count',
                       'taster_count']]
features

,price,age,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count,country_count,province_count,region_count,taster_count
0,13.0,7,33,167,3,5.060606,11.000000,144,23237,12491,2240,7433
1,20.0,7,48,250,4,5.208333,12.000000,144,23237,12491,2240,7433
2,13.0,7,28,136,3,4.857143,9.333333,144,23237,12491,2240,7433
3,20.0,7,39,208,4,5.333333,9.750000,144,23237,12491,2240,7433
4,13.0,7,38,172,3,4.526316,12.666667,144,23237,12491,2240,7433
...,...,...,...,...,...,...,...,...,...,...,...,...
47139,28.0,8,32,186,4,5.812500,8.000000,2,4337,57,5,8
47140,27.0,7,39,222,4,5.692308,9.750000,2,4337,57,5,8
47141,25.0,5,49,256,4,5.224490,12.250000,1,4337,57,5,8
47142,40.0,116,48,233,5,4.854167,9.600000,1,4337,57,5,8


In [26]:
no_price = features.drop(columns='price')
no_price

,age,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count,country_count,province_count,region_count,taster_count
0,7,33,167,3,5.060606,11.000000,144,23237,12491,2240,7433
1,7,48,250,4,5.208333,12.000000,144,23237,12491,2240,7433
2,7,28,136,3,4.857143,9.333333,144,23237,12491,2240,7433
3,7,39,208,4,5.333333,9.750000,144,23237,12491,2240,7433
4,7,38,172,3,4.526316,12.666667,144,23237,12491,2240,7433
...,...,...,...,...,...,...,...,...,...,...,...
47139,8,32,186,4,5.812500,8.000000,2,4337,57,5,8
47140,7,39,222,4,5.692308,9.750000,2,4337,57,5,8
47141,5,49,256,4,5.224490,12.250000,1,4337,57,5,8
47142,116,48,233,5,4.854167,9.600000,1,4337,57,5,8


In [27]:
X = features
y = clean_wines['good_meh']

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=12))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [30]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=0
)

In [31]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

11786/11786 - 1s - loss: 0.3926 - accuracy: 0.8147
Normal Neural Network - Loss: 0.3925867578017576, Accuracy: 0.814695417881012


In [32]:
encoded_predictions = model.predict_classes(X_test_scaled[:])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [33]:
d = {'prediction' : prediction_labels, 
     'actual' : y_test}

In [34]:
predictions_df = pd.DataFrame(data=d)
predictions_df

,prediction,actual
18791,meh,good
10979,good,good
35513,good,good
42181,good,good
17001,good,good
...,...,...
38091,meh,meh
45363,meh,meh
34724,good,good
25429,good,good


In [35]:
predictions_df.to_csv('Data/predictions_df.csv')

In [36]:
X = no_price
y = clean_wines['good_meh']

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=11))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [39]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=0
)

In [40]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

11786/11786 - 1s - loss: 0.4622 - accuracy: 0.7662
Normal Neural Network - Loss: 0.4622319481701756, Accuracy: 0.7661632299423218


In [41]:
encoded_predictions = model.predict_classes(X_test_scaled[:])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [42]:
d = {'prediction' : prediction_labels, 
     'actual' : y_test}

In [43]:
predictions_df = pd.DataFrame(data=d)
predictions_df.sample(30)

,prediction,actual
18640,meh,good
40626,meh,meh
40445,meh,meh
5771,meh,meh
11698,meh,meh
18928,meh,meh
24775,meh,good
12852,meh,meh
1193,meh,meh
29251,meh,meh


In [44]:
predictions_df.to_csv('Data/prediction_df_no_price.csv')

In [45]:
features = clean_wines[['price', 'age', 'word_count', 'char_count', 'sentence_count', 'avg_word_length', 
                        'avg_sentence_lenght', 'winery_count', 'country_count', 'province_count', 'region_count',
                       'taster_count']]
features

,price,age,word_count,char_count,sentence_count,avg_word_length,avg_sentence_lenght,winery_count,country_count,province_count,region_count,taster_count
0,13.0,7,33,167,3,5.060606,11.000000,144,23237,12491,2240,7433
1,20.0,7,48,250,4,5.208333,12.000000,144,23237,12491,2240,7433
2,13.0,7,28,136,3,4.857143,9.333333,144,23237,12491,2240,7433
3,20.0,7,39,208,4,5.333333,9.750000,144,23237,12491,2240,7433
4,13.0,7,38,172,3,4.526316,12.666667,144,23237,12491,2240,7433
...,...,...,...,...,...,...,...,...,...,...,...,...
47139,28.0,8,32,186,4,5.812500,8.000000,2,4337,57,5,8
47140,27.0,7,39,222,4,5.692308,9.750000,2,4337,57,5,8
47141,25.0,5,49,256,4,5.224490,12.250000,1,4337,57,5,8
47142,40.0,116,48,233,5,4.854167,9.600000,1,4337,57,5,8


In [46]:
X = features
y = clean_wines['type']

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=12))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [49]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=0
)

In [50]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

11786/11786 - 1s - loss: 0.4340 - accuracy: 0.8053
Normal Neural Network - Loss: 0.43403116545733017, Accuracy: 0.8052774667739868


In [51]:
encoded_predictions = model.predict_classes(X_test_scaled[:])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [52]:
d = {'prediction' : prediction_labels, 
     'actual' : y_test}

In [53]:
predictions_df = pd.DataFrame(data=d)
predictions_df.sample(30)

,prediction,actual
30729,red,white
2051,red,red
15549,red,red
12723,red,red
19699,red,red
25901,red,white
25003,red,red
10984,red,red
30122,white,red
38952,white,white


In [54]:
white_prd = predictions_df[predictions_df['prediction'] == 'white']
white_prd

,prediction,actual
38760,white,white
422,white,red
7771,white,white
15852,white,red
39005,white,white
...,...,...
33115,white,white
15669,white,red
32346,white,red
38885,white,white


In [55]:
red_prd = predictions_df[predictions_df['prediction'] == 'red']
red_prd

,prediction,actual
18791,red,red
10979,red,red
35513,red,white
42181,red,red
17001,red,red
...,...,...
43377,red,red
38091,red,white
45363,red,red
25429,red,white


In [56]:
white_test_wines = predictions_df[predictions_df['actual'] == 'white']
white_test_wines.

SyntaxError: invalid syntax (<ipython-input-56-0b1919fc91c5>, line 2)

In [ ]:
# X = features
# y = clean_wines['variety']

In [ ]:
# clean_wines['variety'].nunique()

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from tensorflow.keras.utils import to_categorical

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, random_state=1)
# X_scaler = MinMaxScaler().fit(X_train)
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)


# # Step 1: Label-encode data set
# label_encoder = LabelEncoder()
# label_encoder.fit(y_train)
# encoded_y_train = label_encoder.transform(y_train)
# encoded_y_test = label_encoder.transform(y_test)

# # Step 2: Convert encoded labels to one-hot-encoding
# y_train_categorical = to_categorical(encoded_y_train)
# y_test_categorical = to_categorical(encoded_y_test)